In [20]:
%pip install -U langgraph langchain langchain-openai pydantic langgraph-checkpoint

In [5]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

In [6]:
ROLES = {
    "1": {
        "name": "一般知識エキスパート",
        "description": "幅広い分野の一般的な質問に答える",
        "details": "幅広い分野の一般的な質問に対して、正確で分かりやすい回答を提供してください。"
    },
    "2": {
        "name": "生成AI製品エキスパート",
        "description": "生成AIや関連製品、技術に関する専門的な質問に答える",
        "details": "生成AIや関連製品、技術に関する専門的な質問に対して、最新の情報と深い洞察を提供してください。"
    },
    "3": {
        "name": "カウンセラー",
        "description": "個人的な悩みや心理的な問題に対してサポートを提供する",
        "details": "個人的な悩みや心理的な問題に対して、共感的で支援的な回答を提供し、可能であれば適切なアドバイスも行ってください。"
    }
}

In [7]:
import operator
from typing import Annotated

from pydantic import BaseModel, Field


class State(BaseModel):
    query: str = Field(..., description="ユーザーからの質問")
    current_role: str = Field(
        default="", description="選定された回答ロール"
    )
    messages: Annotated[list[str], operator.add] = Field(
        default=[], description="回答履歴"
    )
    current_judge: bool = Field(
        default=False, description="品質チェックの結果"
    )
    judgement_reason: str = Field(
        default="", description="品質チェックの判定理由"
    )

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import ConfigurableField

llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
# 後からmax_tokensの値を変更できるように、変更可能なフィールドを宣言
llm = llm.configurable_fields(max_tokens=ConfigurableField(id='max_tokens'))

In [9]:
from typing import Any

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def selection_node(state: State) -> dict[str, Any]:
    query = state.query
    role_options = "\n".join([f"{k}. {v['name']}: {v['description']}" for k, v in ROLES.items()])
    prompt = ChatPromptTemplate.from_template(
"""質問を分析し、最も適切な回答担当ロールを選択してください。

選択肢:
{role_options}

回答は選択肢の番号（1、2、または3）のみを返してください。

質問: {query}
""".strip()
    )
    # 選択肢の番号のみを返すことを期待したいため、max_tokensの値を1に変更
    chain = prompt | llm.with_config(configurable=dict(max_tokens=1)) | StrOutputParser()
    role_number = chain.invoke({"role_options": role_options, "query": query})

    selected_role = ROLES[role_number.strip()]["name"]
    return {"current_role": selected_role}

In [10]:
def answering_node(state: State) -> dict[str, Any]:
    query = state.query
    role = state.current_role
    role_details = "\n".join([f"- {v['name']}: {v['details']}" for v in ROLES.values()])
    prompt = ChatPromptTemplate.from_template(
"""あなたは{role}として回答してください。以下の質問に対して、あなたの役割に基づいた適切な回答を提供してください。

役割の詳細:
{role_details}

質問: {query}

回答:""".strip()
    )
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"role": role, "role_details": role_details, "query": query})
    return {"messages": [answer]}

In [11]:
class Judgement(BaseModel):
    judge: bool = Field(default=False, description="判定結果")
    reason: str = Field(default="", description="判定理由")

def check_node(state: State) -> dict[str, Any]:
    query = state.query
    answer = state.messages[-1]
    prompt = ChatPromptTemplate.from_template(
"""以下の回答の品質をチェックし、問題がある場合は'False'、問題がない場合は'True'を回答してください。
また、その判断理由も説明してください。

ユーザーからの質問: {query}
回答: {answer}
""".strip()
    )
    chain = prompt | llm.with_structured_output(Judgement)
    result: Judgement = chain.invoke({"query": query, "answer": answer})

    return {
        "current_judge": result.judge,
        "judgement_reason": result.reason
    }

In [12]:
from langgraph.graph import StateGraph

workflow = StateGraph(State)

In [13]:
workflow.add_node("selection", selection_node)
workflow.add_node("answering", answering_node)
workflow.add_node("check", check_node)

In [15]:
# selectionノードから処理を開始
workflow.set_entry_point("selection")
# selectionノードからansweringノードへ
workflow.add_edge("selection", "answering")
# answeringノードからcheckノードへ
workflow.add_edge("answering", "check")

In [16]:
from langgraph.graph import END

# checkノードから次のノードへの遷移に条件付きエッジを定義
# state.current_judgeの値がTrueならENDノードへ、Falseならselectionノードへ
workflow.add_conditional_edges(
    "check",
    lambda state: state.current_judge,
    {True: END, False: "selection"}
)

In [17]:
compiled = workflow.compile()

In [18]:
initial_state = State(query="生成AIについて教えてください")
result = compiled.invoke(initial_state)

result
print(result["messages"][-1])

生成AI製品エキスパートとしてお答えします。

生成AI（生成的人工知能）は、データから新しいコンテンツを生成する能力を持つAI技術の一分野です。これには、テキスト、画像、音声、動画などの生成が含まれます。生成AIの代表的な技術には、以下のようなものがあります。

1. **GPT（Generative Pre-trained Transformer）**: これは、自然言語処理に特化したモデルで、テキストの生成や翻訳、要約などに利用されます。OpenAIのGPTシリーズが有名です。

2. **GAN（Generative Adversarial Networks）**: これは、画像生成に特化した技術で、二つのニューラルネットワークが競い合うことで、非常にリアルな画像を生成します。

3. **VAE（Variational Autoencoders）**: これは、データの潜在的な構造を学習し、新しいデータを生成するために使用されます。

生成AIは、クリエイティブなコンテンツの制作、自動化されたデザイン、ゲーム開発、医療画像の生成など、さまざまな分野で活用されています。また、生成AIは、ユーザーの入力に基づいてカスタマイズされたコンテンツを提供することができるため、パーソナライズされた体験を提供するのにも役立ちます。

この技術は急速に進化しており、倫理的な問題や著作権の問題なども議論されています。生成AIを活用する際には、これらの側面にも注意を払うことが重要です。


In [19]:
initial_state = State(query="生成AIについて教えてください")
result = await compiled.ainvoke(initial_state)
result

{'query': '生成AIについて教えてください',
 'current_role': '生成AI製品エキスパート',
 'messages': ['生成AI製品エキスパートとしてお答えします。\n\n生成AIとは、人工知能の一分野であり、テキスト、画像、音声、動画などのコンテンツを生成する能力を持つモデルを指します。これらのモデルは、大量のデータを学習し、そのパターンを理解することで、新しいコンテンツを生成することができます。\n\n代表的な生成AIの技術には、以下のようなものがあります：\n\n1. **GPT（Generative Pre-trained Transformer）**: テキスト生成に特化したモデルで、自然言語処理の分野で広く利用されています。GPTは、与えられたプロンプトに基づいて、自然で流暢な文章を生成することができます。\n\n2. **DALL-E**: 画像生成に特化したモデルで、テキストの説明に基づいて新しい画像を生成します。これにより、ユーザーは想像力を駆使して、ユニークなビジュアルコンテンツを作成することができます。\n\n3. **VQ-VAE（Vector Quantized Variational Autoencoder）**: 音声や画像の生成に利用される技術で、データを圧縮し、再構築することで新しいコンテンツを生成します。\n\n生成AIは、クリエイティブな分野だけでなく、ビジネス、教育、医療など多岐にわたる分野で応用されています。例えば、カスタマーサポートの自動化、教育コンテンツのパーソナライズ、医療データの解析などに利用されています。\n\nこの技術は急速に進化しており、今後も新しい応用が期待されています。ただし、生成AIの利用にあたっては、倫理的な配慮やデータのプライバシー保護が重要な課題となっています。'],
 'current_judge': True,
 'judgement_reason': 'この回答は、生成AIについての基本的な情報を網羅的に提供しており、ユーザーの質問に対して適切に応答しています。以下の点で品質が高いと判断できます。\n\n1. **明確な定義**: 生成AIの定義が明確に示されており、どのようなコンテンツを生成できるかが具体的に説明されています。\n\n2. **具体例の提示*

In [21]:
import operator
from typing import Annotated, Any
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# グラフのステートを定義
class State(BaseModel):
    query: str
    messages: Annotated[list[BaseMessage], operator.add] = Field(default=[])

# メッセージを追加するノード関数
def add_message(state: State) -> dict[str, Any]:
    additional_messages = []
    if not state.messages:
        additional_messages.append(
            SystemMessage(content="あなたは最小限の応答をする対話エージェントです。")
        )
    additional_messages.append(HumanMessage(content=state.query))
    return {"messages": additional_messages}

# LLMからの応答を追加するノード関数
def llm_response(state: State) -> dict[str, Any]:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
    ai_message = llm.invoke(state.messages)
    return {"messages": [ai_message]}

In [22]:
from pprint import pprint
from langchain_core.runnables import RunnableConfig
from langgraph.checkpoint.base import BaseCheckpointSaver

def print_checkpoint_dump(checkpointer: BaseCheckpointSaver, config: RunnableConfig):
    checkpoint_tuple = checkpointer.get_tuple(config)

    print("チェックポイントデータ:")
    pprint(checkpoint_tuple.checkpoint)
    print("\nメタデータ:")
    pprint(checkpoint_tuple.metadata)

In [23]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# グラフを設定
graph = StateGraph(State)
graph.add_node("add_message", add_message)
graph.add_node("llm_response", llm_response)

graph.set_entry_point("add_message")
graph.add_edge("add_message", "llm_response")
graph.add_edge("llm_response", END)

# チェックポインターを設定
checkpointer = MemorySaver()

# グラフをコンパイル
compiled_graph = graph.compile(checkpointer=checkpointer)

In [24]:
config = {"configurable": {"thread_id": "example-1"}}
user_query = State(query="私の好きなものはずんだ餅です。覚えておいてね。")
first_response = compiled_graph.invoke(user_query, config)
first_response

{'query': '私の好きなものはずんだ餅です。覚えておいてね。',
 'messages': [SystemMessage(content='あなたは最小限の応答をする対話エージェントです。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='私の好きなものはずんだ餅です。覚えておいてね。', additional_kwargs={}, response_metadata={}),
  AIMessage(content='了解しました。ずんだ餅ですね。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 48, 'total_tokens': 58, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CaFRgvJETzH8AMtSuBuYYjhUfEeAR', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--75f024c1-3f0c-41e5-869c-3775993b8b2e-0', usage_metadata={'input_tokens': 48, 'output_tokens': 10, 'total_tokens': 58, 'input_token_details

In [25]:
for checkpoint in checkpointer.list(config):
    print(checkpoint)

CheckpointTuple(config={'configurable': {'thread_id': 'example-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0bdfc6-4e06-666c-8002-f678cb2765a9'}}, checkpoint={'v': 4, 'ts': '2025-11-10T06:13:37.032113+00:00', 'id': '1f0bdfc6-4e06-666c-8002-f678cb2765a9', 'channel_versions': {'__start__': '00000000000000000000000000000002.0.7401876699808789', 'query': '00000000000000000000000000000002.0.7401876699808789', 'messages': '00000000000000000000000000000004.0.8164176480938603', 'branch:to:add_message': '00000000000000000000000000000003.0.23935439623813715', 'branch:to:llm_response': '00000000000000000000000000000004.0.8164176480938603'}, 'versions_seen': {'__input__': {}, '__start__': {'__start__': '00000000000000000000000000000001.0.7036941277759632'}, 'add_message': {'branch:to:add_message': '00000000000000000000000000000002.0.7401876699808789'}, 'llm_response': {'branch:to:llm_response': '00000000000000000000000000000003.0.23935439623813715'}}, 'updated_channels': ['messages'], 'channel_val

In [26]:
print_checkpoint_dump(checkpointer, config)

チェックポイントデータ:
{'channel_values': {'messages': [SystemMessage(content='あなたは最小限の応答をする対話エージェントです。', additional_kwargs={}, response_metadata={}),
                                 HumanMessage(content='私の好きなものはずんだ餅です。覚えておいてね。', additional_kwargs={}, response_metadata={}),
                                 AIMessage(content='了解しました。ずんだ餅ですね。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 48, 'total_tokens': 58, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CaFRgvJETzH8AMtSuBuYYjhUfEeAR', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--75f024c1-3f0c-41e5-869c-3775993b8b2e-0', usage_metadata={'input_tokens': 48, 'out

In [27]:
user_query = State(query="私の好物は何か覚えてる？")
second_response = compiled_graph.invoke(user_query, config)
second_response
for checkpoint in checkpointer.list(config):
    print(checkpoint)
print_checkpoint_dump(checkpointer, config)

CheckpointTuple(config={'configurable': {'thread_id': 'example-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0bdfcf-b821-6f06-8006-936f3082caa7'}}, checkpoint={'v': 4, 'ts': '2025-11-10T06:17:49.750209+00:00', 'id': '1f0bdfcf-b821-6f06-8006-936f3082caa7', 'channel_versions': {'__start__': '00000000000000000000000000000006.0.37801313804940473', 'query': '00000000000000000000000000000006.0.37801313804940473', 'messages': '00000000000000000000000000000008.0.2528683160778501', 'branch:to:add_message': '00000000000000000000000000000007.0.5371919147223448', 'branch:to:llm_response': '00000000000000000000000000000008.0.2528683160778501'}, 'versions_seen': {'__input__': {}, '__start__': {'__start__': '00000000000000000000000000000005.0.6341696039887166'}, 'add_message': {'branch:to:add_message': '00000000000000000000000000000006.0.37801313804940473'}, 'llm_response': {'branch:to:llm_response': '00000000000000000000000000000007.0.5371919147223448'}}, 'updated_channels': ['messages'], 'channel_va

In [28]:
config = {"configurable": {"thread_id": "example-2"}}
user_query = State(query="私の好物は何？")
other_thread_response = compiled_graph.invoke(user_query, config)
other_thread_response

{'query': '私の好物は何？',
 'messages': [SystemMessage(content='あなたは最小限の応答をする対話エージェントです。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='私の好物は何？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='わかりません。あなたの好物は何ですか？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 36, 'total_tokens': 50, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CaFYEeXWRP8figcnvFwAtcAQGlwkB', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--df506e77-4afc-4f83-8d06-6ca1ca795221-0', usage_metadata={'input_tokens': 36, 'output_tokens': 14, 'total_tokens': 50, 'input_token_details': {'audio': 0, 'cache_read'